## Part 1: Custom hash functions

Run the cell below once. 
In the cell below, some helper functions can be found regarding the first task. You don't have to modify them. 

In [2]:
import random
import statistics
import hashlib

def floyd(f: callable, x0: str, limit_iterations: int = 100000) -> int:
    """
    You don't have to modify or call this function.
    Floyd's cycle detection algorithm: https://en.wikipedia.org/wiki/Cycle_detection
    Used to test the quality of the hash function.
    """
    tortoise = f(x0) 
    hare = f(f(x0))

    steps_taken = 0
    while tortoise != hare:
        steps_taken += 1
        tortoise = f(tortoise)
        hare = f(f(hare))

        if steps_taken > limit_iterations:
            return -1
   
    mu = 0
    tortoise = x0
    while tortoise != hare:
        tortoise = f(tortoise)
        hare = f(hare) 
        mu += 1
 
    lam = 1
    hare = f(tortoise)
    while tortoise != hare:
        hare = f(hare)
        lam += 1

    return lam

def test_hash_function(function_to_test: callable):
    """
    You don't have to modify this function.
    Different toy functions that test the quality of a hash function.
    """
    def generate_test_data(n_samples=100000, max_int=2**128):
        inputs = [str(random.randint(0, max_int)) for _ in range(n_samples)]
        outputs = [function_to_test(x) for x in inputs]
        return inputs, outputs

    def test_n_collisions(inputs, outputs):
        print(f'Testing number of inputs with collision. [0%;100%], lower is better, <1% is good.')
        collision_fraction = len(set(outputs)) / len(inputs)
        print(f'\t- Result: {(1 - collision_fraction) * 100:.5f}%')
    
    def test_average_cycle_length(inputs, limit_iterations=50000):
        print(f'Testing number of short cycles. [0; {len(inputs)}] less is better, <20 is good.')
        lengths = [floyd(function_to_test, x, limit_iterations) for x in inputs]
        lengths = [x for x in lengths if x != -1]
        n_cycled = len(lengths)
        print(f'\t- Result: {n_cycled}')
        print(f'Average cycle length. [0; {limit_iterations}], higher is better, >10000 is good.')
        print(f'\t- Result: {int(statistics.mean(lengths))}')

    def test_hamming_weight(outputs):
        def count_hamming_weight(num: str):
            return sum([num & (1<<i)>0 for i in range(32)])
        print(f'Computing average hamming weight: [0;32] closer to 16 is better, 14-18 is good.')
        weights = [count_hamming_weight(int(x, 16)) for x in outputs]
        print(f'\t- Result: {int(statistics.mean(weights))}')
        
    inputs, outputs = generate_test_data()
    test_n_collisions(inputs, outputs)
    test_average_cycle_length(inputs[:20])
    test_hamming_weight(outputs)

## Task 1: Brewing custom hash function

- Your only task is to invent your own cryptographic hash function by modifying the method `custom_hash_function()` in the cell below.
- You can use the following operators:
    - Modular exponentiation: `pow(base, exponent, modulo)`
    - Multiplication with constant integer `inpt * constant_integer`
    - Integer divison with constant integer: `inpt // constant_integer`
    - Bitwise xor with constant integer: `inpt ^ constant_integer`
    - Bitwise and with constant integer: `inpt & constant_integer`
    - Bit complement: `~inpt`
    - Anything else you like
- There is also a wrapper `custom_hash_function_wrapper()`. You don't have to modify it. The wrapper will:
    - Automagically encode string as integer
    - Apply your hash function on the integer
    - **Truncate the result into 4 bytes**
    - Automagically decode the integer into hex string
- Running `test_hash_function(custom_hash_function_wrapper)` will test your function. Below, you can also see the same tests for truncated SHA-256 function. As SHA-256 is a strong cryptographic hash function, your function should strive for similar results in the tests.


In [ ]:
# TODO: Modify the function below to compute a hash of an integer.
def custom_hash_function(inpt: int) -> int:
    return inpt % 1234 # TODO: Replace this example with some real computation


def custom_hash_function_wrapper(inpt: str) -> str:
    """
    You don't have to modify this function.
    Just a wrapper for your function. Accepts string on input, returns hex() representation of the hash.
    """
    inpt = int.from_bytes(inpt.encode(), byteorder='big', signed=False)
    output = custom_hash_function(inpt)
    output = output % 4294967296
    return output.to_bytes(length=4, byteorder='big', signed=False).hex()

test_hash_function(custom_hash_function_wrapper)

Below, the truncated version of SHA-256 is tested, your hash function should get as close to these values as possible

In [3]:
def sha256_32_bits(message: str) -> str:
    """
    You don't have to modify this function.
    Given a string, will compute sha256 of the string and return first 32 bits of the digest
    """
    h = hashlib.sha256()
    h.update(message.encode())
    return h.digest()[:4].hex()

test_hash_function(sha256_32_bits)

Testing number of inputs with collision. [0%;100%], lower is better, <1% is good.
	- Result: 0.00300%
Testing number of short cycles. [0; 20] less is better, <20 is good.
	- Result: 8
Average cycle length. [0; 50000], higher is better, >10000 is good.
	- Result: 26530
Computing average hamming weight: [0;32] closer to 16 is better, 14-18 is good.
	- Result: 16


## Part 2: Password cracking

Again, starting with some helper functions that you don't have to modify

In [4]:
from datetime import datetime

PASSWORD_DATABASE = {'WalterJohns@teleworm.eu': '0c6794',  
                     'AlvinJones@armyspy.com':  '0f0545',  
                     'GenevaClay@armyspy.com':  '094144',  
                     'JuanaJensen@gmail.com':   '0b04b6',  
                     'ClydeTejeda@seznam.cz':   '0fe3f9'}


def authenticate(mail: str, password: str) -> str:
    """
    You don't have to modify this function.
    Given login pair (mail, password), this function will attempt to authenticate the given user
    """
    if mail not in PASSWORD_DATABASE:
        return '[NOK] Incorrect mail'
    elif PASSWORD_DATABASE[mail] == sha256_20_bits(password):
        return f'[OK] Successfully authenticated as {mail}'
    else:
        return f'[NOK] Wrong password.'


def increment_string(arr: str) -> str:
    """
    You don't have to modify this function.
    will increment a string array (taking into account only the a-z, A-Z, 0-9 characters)
    """
    LOWEST_NUMBER = 48
    HIGHEST_NUMER = 57
    LOWEST_UPPECASE_CHAR = 65
    HIGHEST_UPPERCASE_CHAR = 90
    LOWEST_LOWERCASE_CHAR = 98
    HIGHEST_LOWERCASE_CHAR = 122
    
    def increment_alphabet(b: int) -> int:
        """
        You don't have to modify or directly call this function.
        Will increment ascii integer.
        """
        if b == HIGHEST_NUMER:
            return LOWEST_UPPECASE_CHAR
        elif b == HIGHEST_UPPERCASE_CHAR:
            return LOWEST_LOWERCASE_CHAR
        elif b == HIGHEST_LOWERCASE_CHAR:
            return LOWEST_NUMBER
        elif b == 0:
            return LOWEST_NUMBER
        else:
            return b + 1
    
    arr = bytearray(arr.encode())
    for i in range(len(arr) - 1, -1, -1):
        arr[i] = increment_alphabet(arr[i])
        if arr[i] != LOWEST_NUMBER:
            break
    return bytes(arr).decode()

def sha256_20_bits(message: str) -> str:
    """
    You don't have to modify this function.
    Given a string, computes sha256 on it and returns hex representation of first 20 bits of the digest
    """
    h = hashlib.sha256()
    h.update(message.encode())
    dgst = int.from_bytes(h.digest()[:3], byteorder = 'big', signed=False) & int('0FFFFF', 16)
    return dgst.to_bytes(3, byteorder='big', signed=False).hex()

## Task 2: Password cracking

- Your task is to implement two functions below: `find_preimage()` and `attack_database()` to authenticate to the database.
- The `PASSWORD_DATABASE` is in the cell above and passwords were hashed using `sha256_20_bits()` function above.
- `find_preimage(h_x)` should, given a hash `h_x` return some `y` for which `sha256_20_bits(y) = h_x`.
    - It may be useful to use function `increment_string()` to generate different preimage candidates. E.g. calling `increment_string("abc")` will return `"abd"`.
- `attack_database()` should then only call the `find_preimage()` on all database hashes and get a viable password for each mail.
- Using obtained passwords, you can try to call `authenticate(mail, your_found_preimage)` to login to the database.
- Do you think you found the same password as was used to create the database? Or could it be different?.

In [ ]:
def find_preimage(template_hash: str) -> str:
    """
    Given a template_hash (hex encoded string), the function should return y such that sha256_20_bits(y) == template_hash
    """
    pass  # TODO: Implement me.


def attack_database():
    """
    For each entry (mail, password_hash) in PASSWORD_DATABASE, this function should find a suitable preimage and perform authentication
    into the database
    """
    for mail, password_hash in PASSWORD_DATABASE.items():
        pass  # TODO: Implement me.

attack_database()

## Task 2 Bonus

Below is a snippet that creates a password database using your custom hash function from the first part. Try to attack your custom database again and answer the following questions:
- Was cracking 4-byte hashes of your function easier or more difficult than cracking 2 bytes of SHA-256? 
- Did you find shorter or longer passwords then shown below? 
- When using weak hash function, does strong password actually help?

In [ ]:
CUSTOM_FUNCTION_PASSWORD_DATABASE = {'WalterJohns@teleworm.eu': custom_hash_function_wrapper('tackling plausible snort guzzler'),  
                                     'AlvinJones@armyspy.com':  custom_hash_function_wrapper('shifting rush stew rework'),  
                                     'GenevaClay@armyspy.com':  custom_hash_function_wrapper('hastiness percolate suds hurray'),  
                                     'JuanaJensen@gmail.com':   custom_hash_function_wrapper('boundless washstand mud macaroni'),  
                                     'ClydeTejeda@seznam.cz':   custom_hash_function_wrapper('twerp passable music subscript')}

def custom_function_authenticate(mail: str, password: str) -> str:
    """
    You don't have to modify this function.
    Will authenticate using your custom database.
    """
    if mail not in CUSTOM_FUNCTION_PASSWORD_DATABASE:
        return '[NOK] Incorrect mail'
    elif CUSTOM_FUNCTION_PASSWORD_DATABASE[mail] == custom_hash_function_wrapper(password):
        return f'[OK] Successfully authenticated as {mail}'
    else:
        return f'[NOK] Wrong password.'

def custom_function_find_preimage(template_hash: str) -> str:
    """
    Given a template_hash (hex encoded string), the function should return y such that custom_hash_function_wrapper(y) == template_hash
    """
    pass  # TODO: Implement me.


def custom_function_attack_database():
    """
    For each entry (mail, password_hash) in CUSTOM_FUNCTION_PASSWORD_DATABASE, this function should find a suitable preimage and perform authentication
    into the database
    """
    for mail, password_hash in CUSTOM_FUNCTION_PASSWORD_DATABASE.items():
        pass  # TODO: Implement me.

custom_function_attack_database()